In [361]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.patches import Circle, Rectangle, Arc
from sklearn import mixture
from sklearn import ensemble
from sklearn import model_selection
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import log_loss
import time
import itertools
import operator
from sklearn.ensemble import RandomForestClassifier

plt.rcParams["font.family"] = 'NanumBarunGothic'

In [362]:
data = pd.read_pickle('data.pkl')
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,game_date_DT,seconds_from_period_end,seconds_from_period_start,seconds_from_game_start,shot_class,shotLocationCluster,buzzer_beater,quarter,quarter_4th,quarter_5th
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2000-10-31,627,93,93,Jump Shot,4,0,1st,0,0
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2000-10-31,622,98,98,Jump Shot,5,0,1st,0,0
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2000-10-31,465,255,255,Jump Shot,0,0,1st,0,0
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2000-10-31,412,308,308,Jump Shot,12,0,1st,0,0
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2000-10-31,379,341,1061,Good Dunk Shot,1,0,2nd,0,0


In [363]:
data.iloc[:,0:10].head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2


In [364]:
data.iloc[:,10:20].head()

,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,game_date
0,0,2000-01,27,18,NaN,2PT Field Goal,Right Side(R),Middle Shot,16-24 ft.,2000-10-31
1,0,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Middle Shot,8-16 ft.,2000-10-31
2,0,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Middle Shot,16-24 ft.,2000-10-31
3,0,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Middle Shot,16-24 ft.,2000-10-31
4,0,2000-01,19,0,1.0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,2000-10-31


In [365]:
data.iloc[:,20:40].head()

,opponent,shot_id,hometeam,shot_order_in_a_game,game_date_DT,seconds_from_period_end,seconds_from_period_start,seconds_from_game_start,shot_class,shotLocationCluster,buzzer_beater,quarter,quarter_4th,quarter_5th
0,POR,1,0,1,2000-10-31,627,93,93,Jump Shot,4,0,1st,0,0
1,POR,2,0,2,2000-10-31,622,98,98,Jump Shot,5,0,1st,0,0
2,POR,3,0,3,2000-10-31,465,255,255,Jump Shot,0,0,1st,0,0
3,POR,4,0,4,2000-10-31,412,308,308,Jump Shot,12,0,1st,0,0
4,POR,5,0,5,2000-10-31,379,341,1061,Good Dunk Shot,1,0,2nd,0,0


In [366]:
data.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'game_date', 'opponent', 'shot_id', 'hometeam', 'shot_order_in_a_game',
       'game_date_DT', 'seconds_from_period_end', 'seconds_from_period_start',
       'seconds_from_game_start', 'shot_class', 'shotLocationCluster',
       'buzzer_beater', 'quarter', 'quarter_4th', 'quarter_5th'],
      dtype='object')

In [367]:
data.dtypes

action_type                          object
combined_shot_type                   object
game_event_id                         int64
game_id                               int64
lat                                 float64
loc_x                                 int64
loc_y                                 int64
lon                                 float64
minutes_remaining                     int64
period                                int64
playoffs                              int64
season                               object
seconds_remaining                     int64
shot_distance                         int64
shot_made_flag                      float64
shot_type                            object
shot_zone_area                       object
shot_zone_basic                      object
shot_zone_range                      object
game_date                            object
opponent                             object
shot_id                               int64
hometeam                        

In [368]:
data['buzzer_beater'] = data['buzzer_beater'].astype('category')

In [369]:
data['playoffs'] = data['playoffs'].astype('category')

In [370]:
data['hometeam'] = data['hometeam'].astype('category')

In [373]:
data['quarter_4th'] = data['quarter_4th'].astype('category')

In [374]:
data['quarter_5th'] = data['quarter_5th'].astype('category')

In [375]:
data.dtypes

action_type                          object
combined_shot_type                   object
game_event_id                         int64
game_id                               int64
lat                                 float64
loc_x                                 int64
loc_y                                 int64
lon                                 float64
minutes_remaining                     int64
period                                int64
playoffs                           category
season                               object
seconds_remaining                     int64
shot_distance                         int64
shot_made_flag                      float64
shot_type                            object
shot_zone_area                       object
shot_zone_basic                      object
shot_zone_range                      object
game_date                            object
opponent                             object
shot_id                               int64
hometeam                        

In [376]:
seasons = ['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06',
       '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16']
print(seasons)

['1996-97', '1997-98', '1998-99', '1999-00', '2000-01', '2001-02', '2002-03', '2003-04', '2004-05', '2005-06', '2006-07', '2007-08', '2008-09', '2009-10', '2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16']


In [377]:
data.season = data.season.astype('category')
data.season.cat.set_categories(seasons, inplace = True) 

In [398]:
variables = ['season', 'shot_id', 'loc_x', 'loc_y', 'playoffs', 
             'shot_distance', 'hometeam', 'shot_class', 'buzzer_beater', 'shot_zone_basic', 'quarter_5th',
             'shot_made_flag']
# 포함하지만 모델링에 안 쓰는 변수 season, shot_id, shot_made_flag, loc_x, loc_y, ...

predictor = ['playoffs', 'hometeam', 'shot_class', 'buzzer_beater', 'quarter_5th', 'shot_zone_basic']
no_vars = ['season', 'shot_id', 'loc_x', 'loc_y', 'shot_distance', 'shot_made_flag']
yvar = 'shot_made_flag'

In [399]:
def to_category(data):
    dat = data.copy()
    categorial_cols = ['shot_class', 'shot_zone_basic']
    for cc in categorial_cols:
        dummies = pd.get_dummies(dat[cc])
        dummies = dummies.add_prefix("{}#".format(cc))
        dat.drop(cc, axis=1, inplace=True)
        dat = dat.join(dummies)
    return dat

In [400]:
train_data = data[data['shot_made_flag'].notnull()].reset_index(drop=True)

In [401]:
train_data[variables].head()

,season,shot_id,loc_x,loc_y,playoffs,shot_distance,hometeam,shot_class,buzzer_beater,shot_zone_basic,quarter_5th,shot_made_flag
0,2000-01,2,-157,0,0,15,0,Jump Shot,0,Middle Shot,0,0.0
1,2000-01,3,-101,135,0,16,0,Jump Shot,0,Middle Shot,0,1.0
2,2000-01,4,138,175,0,22,0,Jump Shot,0,Middle Shot,0,0.0
3,2000-01,5,0,0,0,0,0,Good Dunk Shot,0,Restricted Area,0,1.0
4,2000-01,6,-145,-11,0,14,0,Jump Shot,0,Middle Shot,0,0.0


In [402]:
traini = to_category(train_data[variables])
traini.head()

,season,shot_id,loc_x,loc_y,playoffs,shot_distance,hometeam,buzzer_beater,quarter_5th,shot_made_flag,...,shot_class#Good Jump Shot,shot_class#Good Layup Shot,shot_class#Hook Shot,shot_class#Jump Shot,shot_class#Layup Shot,shot_class#Tip Shot,shot_zone_basic#Backcourt,shot_zone_basic#Middle Shot,shot_zone_basic#Point3,shot_zone_basic#Restricted Area
0,2000-01,2,-157,0,0,15,0,0,0,0.0,...,0,0,0,1,0,0,0,1,0,0
1,2000-01,3,-101,135,0,16,0,0,0,1.0,...,0,0,0,1,0,0,0,1,0,0
2,2000-01,4,138,175,0,22,0,0,0,0.0,...,0,0,0,1,0,0,0,1,0,0
3,2000-01,5,0,0,0,0,0,0,0,1.0,...,0,0,0,0,0,0,0,0,0,1
4,2000-01,6,-145,-11,0,14,0,0,0,0.0,...,0,0,0,1,0,0,0,1,0,0


In [403]:
traini.season.value_counts()[seasons]

1996-97     383
1997-98     810
1998-99     765
1999-00    1312
2000-01    1575
2001-02    1708
2002-03    1852
2003-04    1371
2004-05    1127
2005-06    1924
2006-07    1579
2007-08    1819
2008-09    1851
2009-10    1772
2010-11    1521
2011-12    1416
2012-13    1328
2013-14      59
2014-15     593
2015-16     932
Name: season, dtype: int64

In [404]:
traini.dtypes

season                             category
shot_id                               int64
loc_x                                 int64
loc_y                                 int64
playoffs                           category
shot_distance                         int64
hometeam                           category
buzzer_beater                      category
quarter_5th                        category
shot_made_flag                      float64
shot_class#Bank Shot                  uint8
shot_class#Dunk Shot                  uint8
shot_class#Fadeaway Shot              uint8
shot_class#Finger Roll                uint8
shot_class#Good Dunk Shot             uint8
shot_class#Good Hook Shot             uint8
shot_class#Good Jump Shot             uint8
shot_class#Good Layup Shot            uint8
shot_class#Hook Shot                  uint8
shot_class#Jump Shot                  uint8
shot_class#Layup Shot                 uint8
shot_class#Tip Shot                   uint8
shot_zone_basic#Backcourt       

In [405]:
test_data = data[data['shot_made_flag'].isnull()].reset_index(drop=True) # test set which shot_made_flag is NaN

In [406]:
testi = to_category(test_data[variables])
testi.head()

,season,shot_id,loc_x,loc_y,playoffs,shot_distance,hometeam,buzzer_beater,quarter_5th,shot_made_flag,...,shot_class#Good Jump Shot,shot_class#Good Layup Shot,shot_class#Hook Shot,shot_class#Jump Shot,shot_class#Layup Shot,shot_class#Tip Shot,shot_zone_basic#Backcourt,shot_zone_basic#Middle Shot,shot_zone_basic#Point3,shot_zone_basic#Restricted Area
0,2000-01,1,167,72,0,18,0,0,0,NaN,...,0,0,0,1,0,0,0,1,0,0
1,2000-01,8,1,28,0,2,0,0,0,NaN,...,0,0,0,1,0,0,0,0,0,1
2,2000-01,17,0,0,0,0,1,1,0,NaN,...,0,1,0,0,0,0,0,0,0,1
3,2000-01,20,0,0,0,0,1,0,0,NaN,...,0,1,0,0,0,0,0,0,0,1
4,2000-01,33,163,76,0,17,0,0,0,NaN,...,0,0,0,1,0,0,0,1,0,0


In [407]:
testi.season.unique()

[2000-01, 2001-02, 2002-03, 2003-04, 2004-05, ..., 2015-16, 1996-97, 1997-98, 1998-99, 1999-00]
Length: 20
Categories (20, object): [2000-01, 2001-02, 2002-03, 2003-04, ..., 1996-97, 1997-98, 1998-99, 1999-00]

In [408]:
rf = RandomForestClassifier(n_estimators=100, random_state=1024, max_depth=8)

seasons

In [409]:
testi["pred"] = 0
testi.head()

,season,shot_id,loc_x,loc_y,playoffs,shot_distance,hometeam,buzzer_beater,quarter_5th,shot_made_flag,...,shot_class#Good Layup Shot,shot_class#Hook Shot,shot_class#Jump Shot,shot_class#Layup Shot,shot_class#Tip Shot,shot_zone_basic#Backcourt,shot_zone_basic#Middle Shot,shot_zone_basic#Point3,shot_zone_basic#Restricted Area,pred
0,2000-01,1,167,72,0,18,0,0,0,NaN,...,0,0,1,0,0,0,1,0,0,0
1,2000-01,8,1,28,0,2,0,0,0,NaN,...,0,0,1,0,0,0,0,0,1,0
2,2000-01,17,0,0,0,0,1,1,0,NaN,...,1,0,0,0,0,0,0,0,1,0
3,2000-01,20,0,0,0,0,1,0,0,NaN,...,1,0,0,0,0,0,0,0,1,0
4,2000-01,33,163,76,0,17,0,0,0,NaN,...,0,0,1,0,0,0,1,0,0,0


In [410]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?

# Adaptive less seasons to consider
- Worse than total learning

In [417]:
submission = pd.DataFrame()
n = 3
n_seasons = [x for x in range(1, n+1)]
for i, s in enumerate(seasons):
    form_in_n = traini[traini.season == s].copy()
    
    print(s, "season")
    if i > 17:
        n_seasons = n_seasons[0:-2]
    for j in n_seasons:
        if i-j <0:
            break
        else:
            form_in_n = form_in_n.append(traini[traini.season == seasons[i-j]].copy(), ignore_index=True)
    
    X = form_in_n.drop(no_vars, axis=1)
    y = form_in_n[yvar].astype('category')
    model = rf.fit(X, y)
    print("log_loss:", metrics.log_loss(y, model.predict_proba(X)))

    testi_s = testi[testi.season == s].copy()
    X_test = testi_s.drop(no_vars, axis=1).drop("pred", axis=1)
    pred = model.predict_proba(X_test)[:, 1]
    submit = pd.DataFrame()
    submit['shot_id'] = testi_s.shot_id
    submit['shot_made_flag'] = pred
    submission = submission.append(submit, ignore_index=True)

1996-97 season
log_loss: 0.558783840304011
1997-98 season
log_loss: 0.571313159735651
1998-99 season
log_loss: 0.5839044905927914
1999-00 season
log_loss: 0.5942265702318485
2000-01 season
log_loss: 0.5965116693486223
2001-02 season
log_loss: 0.6000128841951194
2002-03 season
log_loss: 0.5973698399875443
2003-04 season
log_loss: 0.5957612142667021
2004-05 season
log_loss: 0.5900576466529653
2005-06 season
log_loss: 0.5933317321600183
2006-07 season
log_loss: 0.5976627392053091
2007-08 season
log_loss: 0.5949966171081452
2008-09 season
log_loss: 0.6036183029337988
2009-10 season
log_loss: 0.6007797163748719
2010-11 season
log_loss: 0.6049818450029996
2011-12 season
log_loss: 0.6159836274219184
2012-13 season
log_loss: 0.6107468567352344
2013-14 season
log_loss: 0.6119464311676849
2014-15 season
log_loss: 0.5803711152173608
2015-16 season
log_loss: 0.5777143741398155


In [113]:
submission=submission.sort_values(by="shot_id")

testi.head(4)

submission = pd.DataFrame()
submission["shot_id"] = testi.shot_id
submission["shot_made_flag"] = testi.pred

In [114]:
submission.head(4)

,shot_id,shot_made_flag
694,1,0.367839
695,8,0.228172
696,17,0.502096
697,20,0.635984


In [115]:
submission.to_csv("sub.csv",index=False) # 703/1117

# lambda learning
- 

In [391]:
import pickle

In [411]:
submission = pd.DataFrame()

for i, s in enumerate(seasons):
    form_in_n = traini[traini.season == s].copy()
    
    print(s, "season")
    
    X = form_in_n.drop(no_vars, axis=1)
    y = form_in_n[yvar].astype('category')
    model = rf.fit(X, y)
    print("log_loss:", metrics.log_loss(y, model.predict_proba(X)))
    pkl_filename = "pickle_model_{}.pkl".format(i)  
    with open(pkl_filename, 'wb') as file:  
        pickle.dump(model, file)

    testi_s = testi[testi.season == s].copy()
    X_test = testi_s.drop(no_vars, axis=1).drop("pred", axis=1)
    pred = model.predict_proba(X_test)[:, 1]
    submit = pd.DataFrame()
    submit['shot_id'] = testi_s.shot_id
    submit['shot_made_flag'] = pred
    submission = submission.append(submit, ignore_index=True)

1996-97 season
log_loss: 0.558783840304011
1997-98 season
log_loss: 0.5596109845214525
1998-99 season
log_loss: 0.577933968835299
1999-00 season
log_loss: 0.5911104849196521
2000-01 season
log_loss: 0.5796254800014964
2001-02 season
log_loss: 0.5853820121791662
2002-03 season
log_loss: 0.5741293406308543
2003-04 season
log_loss: 0.5796796060625852
2004-05 season
log_loss: 0.563580353014944
2005-06 season
log_loss: 0.6006680468128047
2006-07 season
log_loss: 0.5938525815544877
2007-08 season
log_loss: 0.5738649201683733
2008-09 season
log_loss: 0.6044308430828005
2009-10 season
log_loss: 0.5813079704138936
2010-11 season
log_loss: 0.6098015436312775
2011-12 season
log_loss: 0.6144967565202325
2012-13 season
log_loss: 0.5864336441543979
2013-14 season
log_loss: 0.48884047446558626
2014-15 season
log_loss: 0.5813773443482503
2015-16 season
log_loss: 0.5777143741398155


lmda2 = [0.3, 0.7]
lmda3 = [0.1, 0.3, 0.6]
lmda4 = [0.1, 0.2, 0.3, 0.4]
lmda5 = [0.05, 0.1, 0.2, 0.25, 0.4]
lmda6 = [0.02, 0.05, 0.1, 0.16, 0.26, 0.41]

In [412]:
def lmd(n):
    return [np.exp(y/5)/np.sum([np.exp(x/5) for x in range(1,n+1)]) for y in range(1,n+1)]

submission = pd.DataFrame()

for i, s in enumerate(seasons):
    print(s)
    testset = testi[testi.season == s].copy()
    test_X = testset.drop(no_vars, axis=1).drop("pred", axis=1)
    tmp = pd.DataFrame()
    tmp['shot_id'] = testset.shot_id
    
    n = 1
    while i-n >= 0 and n<=6:
        n += 1
    if n==1:
        pkl_filename = "pickle_model_{}.pkl".format(i) 
        with open(pkl_filename, 'rb') as file:  
            pickle_model = pickle.load(file)
        tmp['shot_made_flag'] = lmd(1)*pickle_model.predict_proba(test_X)[:, 1]
    elif n==2:
        tmp['shot_made_flag'] = 0
        for j in range(n):
            pkl_filename = "pickle_model_{}.pkl".format(i-1+j) 
            with open(pkl_filename, 'rb') as file:  
                pickle_model = pickle.load(file)
            tmp['shot_made_flag'] += lmd(2)[j]*pickle_model.predict_proba(test_X)[:, 1]
    elif n==3:
        tmp['shot_made_flag'] = 0
        for j in range(n):
            pkl_filename = "pickle_model_{}.pkl".format(i-2+j) 
            with open(pkl_filename, 'rb') as file:  
                pickle_model = pickle.load(file)
            tmp['shot_made_flag'] += lmd(3)[j]*pickle_model.predict_proba(test_X)[:, 1]
    elif n==4:
        tmp['shot_made_flag'] = 0
        for j in range(n):
            pkl_filename = "pickle_model_{}.pkl".format(i-3+j) 
            with open(pkl_filename, 'rb') as file:  
                pickle_model = pickle.load(file)
            tmp['shot_made_flag'] += lmd(4)[j]*pickle_model.predict_proba(test_X)[:, 1]
    elif n==5:
        tmp['shot_made_flag'] = 0
        for j in range(n):
            pkl_filename = "pickle_model_{}.pkl".format(i-4+j) 
            with open(pkl_filename, 'rb') as file:  
                pickle_model = pickle.load(file)
            tmp['shot_made_flag'] += lmd(5)[j]*pickle_model.predict_proba(test_X)[:, 1]
    elif n==6:
        tmp['shot_made_flag'] = 0
        for j in range(n):
            pkl_filename = "pickle_model_{}.pkl".format(i-5+j) 
            with open(pkl_filename, 'rb') as file:  
                pickle_model = pickle.load(file)
            tmp['shot_made_flag'] += lmd(6)[j]*pickle_model.predict_proba(test_X)[:, 1]
    else:
        tmp['shot_made_flag'] = 0
        for j in range(n):
            pkl_filename = "pickle_model_{}.pkl".format(i-6+j) 
            with open(pkl_filename, 'rb') as file:  
                pickle_model = pickle.load(file)
            tmp['shot_made_flag'] += lmd(7)[j]*pickle_model.predict_proba(test_X)[:, 1]
    submission = submission.append(tmp, ignore_index=True)

In [413]:
submission = pd.DataFrame()

for i, s in enumerate(seasons):
    print(s)
    testset = testi[testi.season == s].copy()
    test_X = testset.drop(no_vars, axis=1).drop("pred", axis=1)
    tmp = pd.DataFrame()
    tmp['shot_id'] = testset.shot_id
    
    n = 1
    while i-n >= 0:
        n += 1
    
    tmp['shot_made_flag'] = 0
    for j in range(n):
        pkl_filename = "pickle_model_{}.pkl".format(i-(n-1)+j) 
        with open(pkl_filename, 'rb') as file:  
            pickle_model = pickle.load(file)
        tmp['shot_made_flag'] += lmd(n)[j]*pickle_model.predict_proba(test_X)[:, 1]
        
    submission = submission.append(tmp, ignore_index=True)

1996-97
1997-98
1998-99
1999-00
2000-01
2001-02
2002-03
2003-04
2004-05
2005-06
2006-07
2007-08
2008-09
2009-10
2010-11
2011-12
2012-13
2013-14
2014-15
2015-16


In [414]:
submission=submission.sort_values(by="shot_id")

testi.head(4)

submission = pd.DataFrame()
submission["shot_id"] = testi.shot_id
submission["shot_made_flag"] = testi.pred

In [415]:
submission.to_csv("sub.csv",index=False) # 703/1117

In [416]:
submission.to_pickle("submission2.pkl")

In [419]:
lmd(2)

[0.4501660026875221, 0.549833997312478]

In [358]:
6/(1+2+3+4+5+6)

0.2857142857142857

In [359]:
2/(1+2+3)

0.3333333333333333

In [216]:
3/(1+2+3)

0.5

In [ ]:
heyday = df.loc[(df.season >= '1998-99') & (df.season <= '2010-11')].reset_index(drop=True)
heyday.head()